In [1]:
#!pip install openpyxl

import openpyxl
import pandas as pd
import os
import re
import numpy as np

On va procéder ici au merging des bases de données. Le but est d'obtenir à la fin une base de données avec :

ID ASSO : numéro de l'association tel que définie dans base de données anti trans

ID ARTICLE : 2 digits pour ID ASSO + 4 digits pour le numéro de l'article en regroupant par ID ASSO

contenu : texte à interpréter

métadonnées : pas toujours récupérées, np.nan si pas récupérées.
- Type de document
- Auteur
- Date
- Titre
- URL
## Préparation des sites 

### Préparation de la base manuelle

In [2]:
# Obtenez le chemin du répertoire parent du répertoire src
base_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Construisez le chemin complet vers votre fichier Excel
fichier_excel = os.path.join(base_dir, 'data', 'text', 'manuel.xlsx')

# Chargez le fichier Excel dans un DataFrame
sites = pd.read_excel(fichier_excel, sheet_name=0)

In [3]:
#On garde les colonnes intéressantes 
sites = sites[['Id', 'nom', 'site', 'Idéologie']]
sites = sites.dropna(subset=['nom'])
sites.head()

,Id,nom,site,Idéologie
0,1,Renaissance Catholique,https://renaissancecatholique.fr,Intégrisme Catholique
1,2,AGRIF,https://www.lagrif.fr/,"Islamophobie, Nationalisme"
2,3,Génération Zemmour,https://www.generation-zemmour.fr,"Nationaliste, Identitaire"
3,4,ISSEP,https://www.issep.fr/,Extrême-Droite
4,5,Egalité et Réconciliation,https://www.egaliteetreconciliation.fr,"Antisémitisme, Identitaire"


In [4]:
manuel1 = pd.read_excel(fichier_excel, sheet_name=1) #Récupération manuel1
manuel1 = manuel1.dropna(subset=['contenu']) #Fuck les NaN
nouveaux_noms = {
    'URL ': 'URL',
}
manuel1["Titre"] = np.nan
manuel1["Auteur"] = np.nan
manuel1["Date"] = np.nan
manuel1["ID ASSO"] = manuel1["ID ASSO"].apply(int)
manuel1.drop(columns=["ID ARTICLE"], inplace=True)
manuel1 = manuel1.rename(columns=nouveaux_noms)
manuel1.head()

,ID ASSO,URL,Type de document,contenu,Titre,Auteur,Date
0,1,https://renaissancecatholique.fr/blog/les-just...,Article d'opinion,"Par jugement du 23 novembre 2021, la XVIIème c...",NaN,NaN,NaN
1,3,https://www.generation-zemmour.fr/le-funeste-i...,Article d'opinion,Le funeste inventaire de la loi bioéthique ado...,NaN,NaN,NaN
2,3,https://www.generation-zemmour.fr/1568,Article d'opinion,"Rousseau, Woerth et les wokes : Quand les Lumi...",NaN,NaN,NaN
3,3,https://www.generation-zemmour.fr/le-feminisme...,Article d'opinion,Le féminisme progressiste actuel défend-il tou...,NaN,NaN,NaN
4,3,https://www.generation-zemmour.fr/le-pronom-ie...,Article d'opinion,Le pronom « iel » ou le sexe des anges\nLa thé...,NaN,NaN,NaN


### Adaptation de Femellistes 

In [5]:
# Obtenez le chemin du répertoire parent du répertoire src
base_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Construisez le chemin complet vers votre fichier Excel
fichier = os.path.join(base_dir, 'data', 'text', 'femelliste_articles.tsv')

# Chargez le fichier Excel dans un DataFrame
femilliste = pd.read_csv(fichier ,sep='\t', encoding='utf-8')

In [6]:
femilliste = femilliste.drop('Catégories', axis=1, errors='ignore')
femilliste['ID ASSO'] = 12 #On ajoute l'ID ASSO poue le merge futur 
femilliste['Type de document'] = 'opinion' #On ajoute l'ID ASSO poue le merge futur 

# Renommer les colonnes pour faciliter la concaténation

nouveaux_noms = {
    'URL Complète': 'URL',
    'Texte Complet': 'contenu',
}
femilliste.drop(columns=["Extrait"],inplace=True)
femilliste = femilliste.rename(columns=nouveaux_noms)
femilliste.head()

,Titre,Auteur,Date,contenu,URL,ID ASSO,Type de document
0,Le transgenrisme instrumentalise la cause inte...,Alice,06/10/2023,Le transgenrisme instrumentalise la cause inte...,https://www.femelliste.com/articles-femellisme...,12,opinion
1,Comment le transgenrisme et la culture queer i...,FEMELLISTE,26/06/2023,Comment le transgenrisme et la culture queer i...,https://www.femelliste.com/articles-femellisme...,12,opinion
2,Entretien avec le collectif “Féminicides par c...,Marguerite Stern,21/04/2023,Entretien avec le collectif “Féminicides par c...,https://www.femelliste.com/articles-femellisme...,12,opinion
3,Tania : mère d’une adolescente anciennement tr...,Marguerite Stern,30/03/2023,Tania : mère d’une adolescente anciennement tr...,https://www.femelliste.com/articles-femellisme...,12,opinion
4,"Laetitia Ky : “Je n'ai pas de genre, j'ai un s...",Marguerite Stern,14/03/2023,"Laetitia Ky : “Je n'ai pas de genre, j'ai un s...",https://www.femelliste.com/articles-femellisme...,12,opinion


### Adaptation de Tradfem

In [7]:
# Obtenez le chemin du répertoire parent du répertoire src
base_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Construisez le chemin complet vers votre fichier Excel
fichier = os.path.join(base_dir, 'data', 'text', 'tradfem_articles.tsv')

# Chargez le fichier Excel dans un DataFrame
tradfem = pd.read_csv(fichier ,sep='\t', encoding='utf-8')

In [8]:
#Ajout et suppressions de colonnes intéressantes 
tradfem = tradfem.drop(['Image URL'], axis=1, errors='ignore')
tradfem['ID ASSO'] = 13 #On ajoute l'ID ASSO poue le merge futur 
tradfem['Type de document'] = 'blog' #On ajoute l'ID ASSO poue le merge futur 

#Rename utiles 
nouveaux_noms = {
    'URL Complète': 'URL',
    'Texte Complet': 'contenu',
}

tradfem = tradfem.rename(columns=nouveaux_noms)
tradfem.head()

,Titre,Date,Auteur,contenu,URL,ID ASSO,Type de document
0,« Accorder priorité à l’identité de genre sur ...,Il y a 13 heures 03/12/2023,TRADFEM,« Accorder priorité à l’identité de genre sur ...,https://tradfem.wordpress.com/2023/12/03/accor...,13,blog
1,Les féminicides se multiplient au Canada… et a...,Il y a 4 jours 30/11/2023,TRADFEM,Les féminicides se multiplient au Canada… et a...,https://tradfem.wordpress.com/2023/11/30/les-f...,13,blog
2,Oscar Pistorius est libéré après avoir purgé m...,Il y a 1 semaine 24/11/2023,TRADFEM,Oscar Pistorius est libéré après avoir purgé m...,https://tradfem.wordpress.com/2023/11/24/oscar...,13,blog
3,« Pourquoi fait-il ça? » est maintenant offert...,17/11/2023,TRADFEM,« Pourquoi fait-il ça? » est maintenant offert...,https://tradfem.wordpress.com/2023/11/17/pourq...,13,blog
4,"L’horreur, l’horreur",12/11/2023,TRADFEM,"L’horreur, l’horreurTRADFEM/12/11/2023________...",https://tradfem.wordpress.com/2023/11/12/lhorr...,13,blog


### Observatoire de la petite sirène

In [9]:
# Obtenez le chemin du répertoire parent du répertoire src
base_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Construisez le chemin complet vers votre fichier Excel
fichier = "../data/text/petitesirene_articles.tsv" # os.path.join(base_dir, 'data', 'text', 'petitesirene_article.tsv')

# Chargez le fichier Excel dans un DataFrame
petitesirene = pd.read_csv(fichier ,sep='\t', encoding='utf-8')

#Ajout et suppressions de colonnes intéressantes 
petitesirene.drop(['Unnamed: 0'], axis=1, errors='ignore',inplace=True)
petitesirene['ID ASSO'] = 23 #On ajoute l'ID ASSO poue le merge futur 
petitesirene['Type de document'] = 'analyse' #On ajoute l'ID ASSO poue le merge futur 

#Rename utiles 
nouveaux_noms = {
        'title': 'Titre',
        'publication_date': 'Date',
        'author': 'Auteur',
        'url': 'URL',
        'text': 'contenu'
}

petitesirene = petitesirene.rename(columns=nouveaux_noms)
petitesirene.head()

,Titre,URL,Date,Auteur,contenu,ID ASSO,Type de document
0,Papier de Jacqueline Schaeffer - n°4,https://www.observatoirepetitesirene.org/post/...,30 janv.,La petite Sirène,Jacqueline Schaeffer est psychanalyste POURQ...,23,analyse
1,Papier de Christian Flavigny - n°3,https://www.observatoirepetitesirene.org/post/...,22 janv.,La petite Sirène,"Christian Flavigny est psychiatre, pédopsychia...",23,analyse
2,Papier de François Rastier - n°2,https://www.observatoirepetitesirene.org/post/...,26 déc. 2023,La petite Sirène,François Rastier est membre du conseil scienti...,23,analyse
3,Papier de Claudio Rubiliani - n°1,https://www.observatoirepetitesirene.org/post/...,11 déc. 2023,La petite Sirène,Claudio Rubiliani est au directoire de l'OPS. ...,23,analyse


## Manuel 2 : PDF

In [10]:
fichier = "../data/text/manuel2.tsv"

manuel2 = pd.read_csv(fichier ,sep='\t', encoding='utf-8')
manuel2["Titre"] = np.nan

manuel2["Auteur"] = np.nan

manuel2.drop(columns=["ID ARTICLE"], inplace=True)
manuel2.head()

,ID ASSO,URL,Type de document,Date,contenu,Titre,Auteur
0,11,https://soseducation.org/docs/notes-etudes-ent...,Note de synthèse,NaN,la question tr ansg enrechez les enfantscessez...,NaN,NaN
1,23,https://www.observatoirepetitesirene.org/post/...,NaN,NaN,1 les traitements hormonaux utilisés pour les...,NaN,NaN
2,22,https://www.observatoirepetitesirene.org/_file...,Rapport médical,NaN,11 filles autistes et identité de genre 8 12 f...,NaN,NaN
3,11,https://soseducation.org/petitions-mobilisatio...,Pétition,NaN,PÉTITION À l’attention de monsieur Gabriel Att...,NaN,NaN
4,11,https://soseducation.org/docs/mobilisations/al...,Lettre,NaN,Monsieur le Ministre de l’Éducation nationale ...,NaN,NaN


## Résistance Lesbienne

In [11]:
# Obtenez le chemin du répertoire parent du répertoire src
base_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Construisez le chemin complet vers votre fichier Excel
fichier = "../data/text/resistance_lesbienne_articles.tsv" # os.path.join(base_dir, 'data', 'text', 'petitesirene_article.tsv')

# Chargez le fichier Excel dans un DataFrame
resistance_lesbienne = pd.read_csv(fichier ,sep='\t', encoding='utf-8')

#Ajout et suppressions de colonnes intéressantes 
resistance_lesbienne['ID ASSO'] = 21 #On ajoute l'ID ASSO poue le merge futur 
resistance_lesbienne['Type de document'] = 'analyse'
resistance_lesbienne['URL'] = np.nan

#Rename utiles 
nouveaux_noms = {
        'title': 'Titre',
        'date': 'Date',
        'author': 'Auteur',
        'text': 'contenu'
}

resistance_lesbienne = resistance_lesbienne.rename(columns=nouveaux_noms)
resistance_lesbienne.head()

,Titre,Date,contenu,Auteur,ID ASSO,Type de document,URL
0,Protégeons nos filles ! [1] La dysphorie de ge...,2021-07-27,Nous publions cet article écrit par une femme ...,résistance lesbienne,21,analyse,NaN
1,Communiqué – Agression lesbophobe à la Marche ...,2021-07-02,Cliquez ici pour le PDF de ce communiqué\nNoue...,résistance lesbienne,21,analyse,NaN
2,Texte de notre tract – Marche des Fiertés Pari...,2021-07-02,"PDF du tract ici\nNoues, lesbiennes féministes...",résistance lesbienne,21,analyse,NaN
3,Pourquoi je suis une femme lesbienne et pas no...,2021-08-27,Une grande mercie à Seshat pour noues avoir so...,résistance lesbienne,21,analyse,NaN
4,Communiqué – Agression lesbophobe à la Marche ...,2021-09-14,Lors de notre action de visibilité lesbienne à...,résistance lesbienne,21,analyse,NaN


## Cry for recognition

In [12]:
# Obtenez le chemin du répertoire parent du répertoire src
base_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Construisez le chemin complet vers votre fichier Excel
fichier = "../data/text/cry_for_recognition_articles.tsv" # os.path.join(base_dir, 'data', 'text', 'petitesirene_article.tsv')

# Chargez le fichier Excel dans un DataFrame
cry_for_recognition = pd.read_csv(fichier ,sep='\t', encoding='utf-8')

#Ajout et suppressions de colonnes intéressantes 
cry_for_recognition['ID ASSO'] = 20 #On ajoute l'ID ASSO poue le merge futur 
cry_for_recognition['Type de document'] = 'analyse'
cry_for_recognition['URL'] = np.nan

#Rename utiles 
nouveaux_noms = {
        'title': 'Titre',
        'date': 'Date',
        'author': 'Auteur',
        'text': 'contenu'
}

cry_for_recognition = cry_for_recognition.rename(columns=nouveaux_noms)
cry_for_recognition.head()

,Auteur,Date,Titre,contenu,ID ASSO,Type de document,URL
0,Cry for Recognition,NaN,EssaiOriginal: Unherd – How we created a self-...,Il fut un temps où l’on devenait une personne ...,20,analyse,NaN
1,Cry for Recognition,NaN,Original: Detransitioner Perspective: What Con...,"Je suis née en 1963. Dès mon adolescence, j’ét...",20,analyse,NaN
2,Cry for Recognition,NaN,Original: PITT – Transition wasn’t appropriate...,"Ces deux derniers jours (et nuits), j’ai réflé...",20,analyse,NaN
3,Cry for Recognition,NaN,Nous remercions Walt Heyer de nous avoir donné...,Walt Heyer a subi une opération de changement ...,20,analyse,NaN
4,Cry for Recognition,NaN,"Genspect – No, trans is not ‘the new gay’",On pense souvent que le fait d’être transgenre...,20,analyse,NaN


## Christine Le Doaré

In [13]:
# Obtenez le chemin du répertoire parent du répertoire src
base_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Construisez le chemin complet vers votre fichier Excel
fichier = "../data/text/christineld_articles.tsv" # os.path.join(base_dir, 'data', 'text', 'petitesirene_article.tsv')

# Chargez le fichier Excel dans un DataFrame
christineld = pd.read_csv(fichier ,sep='\t', encoding='utf-8')

#Ajout et suppressions de colonnes intéressantes 
christineld['ID ASSO'] = 17 #On ajoute l'ID ASSO poue le merge futur 
christineld['Type de document'] = 'analyse'
christineld['URL'] = None

#Rename utiles 
nouveaux_noms = {
        'title': 'Titre',
        'date': 'Date',
        'author': 'Auteur',
        'text': 'contenu'
}

christineld = christineld.rename(columns=nouveaux_noms)
christineld.head()

,Unnamed: 0,Title,Publication Date,Author,URL,Text,ID ASSO,Type de document
0,0,Quel-le présidentiable aura le courage de le d...,2016-12-06T12:19:09+0100,Christine Le Doaré,None,Les gouvernements de gauche ne peuvent tout si...,17,analyse
1,1,Combien y a-t-il de féminismes ?,2016-10-07T14:50:03+0200,Christine Le Doaré,None,"N’en déplaise aux modes médiatiques, aux révis...",17,analyse
2,2,"Burkini, réanimez les féministes !",2016-08-17T16:44:03+0200,Christine Le Doaré,None,Ah les femmes et leurs corps ! En 2016 a-t-on ...,17,analyse
3,3,"Politiques, intellectuels, médias, reprenez-vo...",2016-08-04T17:31:55+0200,Christine Le Doaré,None,Il est temps de prendre conscience de la situa...,17,analyse
4,4,"Orlando, tuerie homophobe ou terrorisme islami...",2016-06-19T11:12:01+0200,Christine Le Doaré,None,Encore un immonde carnage et qui rappelle d’au...,17,analyse


## Merging

In [14]:
df = pd.concat([femilliste,christineld,cry_for_recognition,resistance_lesbienne, manuel1,manuel2, petitesirene,tradfem], ignore_index=True, sort=False)
df.sample(10)

,Titre,Auteur,Date,contenu,URL,ID ASSO,Type de document,Unnamed: 0,Title,Publication Date,Author,Text
346,NaN,NaN,2021-02-01 00:00:00,Homme et femme Il les créa !\r\n\r\nLe monde m...,https://www.civitas-institut.com/2021/02/01/la...,31,Article d'opinion,NaN,NaN,NaN,NaN,NaN
20,"Le féminisme critique du genre, c’est quoi ?",Dora Moutot & Marguerite Stern,18/10/2022,"Le féminisme critique du genre, c’est quoi ?Le...",https://www.femelliste.com/articles-femellisme...,12,opinion,NaN,NaN,NaN,NaN,NaN
263,NaN,NaN,NaN,Le changement terminologique à l’origine de la...,https://cryforrecognition.be/fr/les-garcons-ma...,20,Article d'opinion,NaN,NaN,NaN,NaN,NaN
17,Les pronoms transgenres : un délire mégalomani...,Anissia Docaigne-makhroff,27/10/2022,Les pronoms transgenres : un délire mégalomani...,https://www.femelliste.com/articles-femellisme...,12,opinion,NaN,NaN,NaN,NaN,NaN
172,“Je suis tellement fière d’être lesbienne”,résistance lesbienne,2023-04-26,Noues sommes heureuses de publier une traducti...,NaN,21,analyse,NaN,NaN,NaN,NaN,NaN
258,NaN,NaN,NaN,\r\nDes associations déplorent la judiciarisat...,https://cryforrecognition.be/fr/ne-confondons-...,20,Article d'opinion,NaN,NaN,NaN,NaN,NaN
91,NaN,NaN,NaN,NaN,None,17,analyse,62.0,A condition de marier les FEMEN aux féministes...,2013-03-06T13:56:46+0100,Christine Le Doaré,"Hier soir, j’étais bien installée devant le do..."
417,Nous réapproprier Sinéad O’Connor,TRADFEM,26/07/2023,Nous réapproprier Sinéad O’ConnorTRADFEM/26/07...,https://tradfem.wordpress.com/2023/07/26/nous-...,13,blog,NaN,NaN,NaN,NaN,NaN
384,JULIE BINDEL : Je pensais avoir tout vu…,TRADFEM,07/10/2023,JULIE BINDEL : Je pensais avoir tout vu…TRADFE...,https://tradfem.wordpress.com/2023/10/07/julie...,13,blog,NaN,NaN,NaN,NaN,NaN
198,NaN,NaN,NaN,"À notre avis, la première mission de l’École e...",https://www.debout-la-france.fr/actualite/lors...,24,Article d'opinion,NaN,NaN,NaN,NaN,NaN


In [15]:
#Refaire des nombres de référence des articles
df['ID ARTICLE'] = range(1, len(df) + 1)

# Convert 'ID ASSO' to a 2-digit string
df['ID ASSO Str'] = df['ID ASSO'].apply(lambda x: f"{x:02}")

# Generate a row number within each group, starting from 1, and convert to a 2-digit string
df['Sequence Number'] = (df.groupby('ID ASSO').cumcount() + 1).apply(lambda x: f"{x:04}")

# Concatenate 'ID ASSO Str' and 'Sequence Number' to form 'ID ARTICLE'
df['ID ARTICLE'] = df['ID ASSO Str'] + df['Sequence Number']

df.drop(columns=["ID ASSO Str","Sequence Number"],inplace=True)

# Resulting DataFrame
df.head(10)

,Titre,Auteur,Date,contenu,URL,ID ASSO,Type de document,Unnamed: 0,Title,Publication Date,Author,Text,ID ARTICLE
0,Le transgenrisme instrumentalise la cause inte...,Alice,06/10/2023,Le transgenrisme instrumentalise la cause inte...,https://www.femelliste.com/articles-femellisme...,12,opinion,NaN,NaN,NaN,NaN,NaN,120001
1,Comment le transgenrisme et la culture queer i...,FEMELLISTE,26/06/2023,Comment le transgenrisme et la culture queer i...,https://www.femelliste.com/articles-femellisme...,12,opinion,NaN,NaN,NaN,NaN,NaN,120002
2,Entretien avec le collectif “Féminicides par c...,Marguerite Stern,21/04/2023,Entretien avec le collectif “Féminicides par c...,https://www.femelliste.com/articles-femellisme...,12,opinion,NaN,NaN,NaN,NaN,NaN,120003
3,Tania : mère d’une adolescente anciennement tr...,Marguerite Stern,30/03/2023,Tania : mère d’une adolescente anciennement tr...,https://www.femelliste.com/articles-femellisme...,12,opinion,NaN,NaN,NaN,NaN,NaN,120004
4,"Laetitia Ky : “Je n'ai pas de genre, j'ai un s...",Marguerite Stern,14/03/2023,"Laetitia Ky : “Je n'ai pas de genre, j'ai un s...",https://www.femelliste.com/articles-femellisme...,12,opinion,NaN,NaN,NaN,NaN,NaN,120005
5,"Bloqueurs de puberté : au journal le Monde, pe...",Zest De Gravity,18/02/2023,"Bloqueurs de puberté : au journal le Monde, pe...",https://www.femelliste.com/articles-femellisme...,12,opinion,NaN,NaN,NaN,NaN,NaN,120006
6,Canada : Un lycéen exclu et arrêté par la poli...,Marguerite Stern,13/02/2023,Canada : Un lycéen exclu et arrêté par la poli...,https://www.femelliste.com/articles-femellisme...,12,opinion,NaN,NaN,NaN,NaN,NaN,120007
7,Quand la CAF fait la promotion du transgenrisme.,Marguerite Stern,02/02/2023,Quand la CAF fait la promotion du transgenrism...,https://www.femelliste.com/articles-femellisme...,12,opinion,NaN,NaN,NaN,NaN,NaN,120008
8,États-unis : un homme jugé coupable de féminic...,Marguerite Stern,31/01/2023,États-unis : un homme jugé coupable de féminic...,https://www.femelliste.com/articles-femellisme...,12,opinion,NaN,NaN,NaN,NaN,NaN,120009
9,Royaume-uni : la loi concernant l’incarcératio...,Marguerite Stern,26/01/2023,Royaume-uni : la loi concernant l’incarcératio...,https://www.femelliste.com/articles-femellisme...,12,opinion,NaN,NaN,NaN,NaN,NaN,120010


In [16]:
df = df[["ID ASSO","ID ARTICLE", "contenu", "Type de document","Auteur","Date","Titre","URL"]]
df.head(1)

,ID ASSO,ID ARTICLE,contenu,Type de document,Auteur,Date,Titre,URL
0,12,120001,Le transgenrisme instrumentalise la cause inte...,opinion,Alice,06/10/2023,Le transgenrisme instrumentalise la cause inte...,https://www.femelliste.com/articles-femellisme...


In [17]:
# last operation : we supprimate row with contenu = np.nan

print(f"Length data base articles before ejecting NAs {len(df)}")
df = df.dropna(subset=['contenu'])
print(f"Length data base articles after ejecting NAs {len(df)}") # there was 2 NaN in cry for recognition


Length data base articles before ejecting NAs 489
Length data base articles after ejecting NAs 397


In [18]:
#On exporte 
# Obtenez le chemin du répertoire parent du répertoire src
base_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Construisez le chemin complet vers le répertoire de données
chemin_data = os.path.join(base_dir, 'data/intermediate')

# Enregistrez le DataFrame dans un fichier CSV dans le répertoire data
df.to_csv(os.path.join(chemin_data, 'base_merged.csv'), sep=';', index=False, quotechar='"')